# 09. Docker Container

다룰 내용: 클라이언트 서버 모델, 도커 컨테이너 사용하기

## 1. 클라이언트 서버 모델

* 클라이언트 서버 모델(client-server model)은 서비스의 요청자와 자원 제공자 사이에 작업을 분리해주는 분산 애플리케이션(like 클러스터) 구조, 네트워크 아키텍처를 말한다.
* 클라이언트는 사용자의 입력에 초점을 맞추었고, 서버는 공유 데이터의 처리, 네트워크 관리 등의 역할을 수행한다.
* 도커는 클라이언트 서버 모델을 따르기 때문에 이에 대한 정보를 확인해볼 수 있다. `docker version`
![docker version](fig 10/fig10-1.png)
* 도커는 하나의 실행파일이지만 클라이언트와 서버 역할을 분담하여 각각 명령을 수행한다.
* 도커 커맨드를 입력하면 도커 클라이언트가 도커 서버로 명령을 전송하고, 결과를 터미널에 출력한다.
![docker client-host](fig 10/fig10-2.png)
* 기본적으로 서버는 도커 서버로 지정되어 있기 때문에 도커에서 명령을 내리면 하나의 컴퓨터 프로그램을 작동시키는 것 같지만 사실 가상 서버에 설치된 도커와의 상호작용이 있다.


## 2. 컨테이너 vs 이미지

* 이미지: 컨테이너 실행에 필요한 파일과 설정값 등을 포함하고 있다.
* 컨테이너: `docker run <image>`을 통해 이미지를 실행한 상태이다.
* 하나의 이미지에서 여러 컨테이너를 생성할 수 있다.
* 컨테이너의 상태가 바뀌거나 삭제되더라도 이미지는 그대로 남아있다.
* 이미지는 컨테이너를 실행하기 위한 모든 정보를 가지고 있다.
* 따라서 도커 허브 상에서 다운 받거나 게시하는 것은 이미지이다.
![docker image](fig 10/fig10-3.png)
* 효율적인 작업을 위해 이미지는 유니온 파일 시스템(레이어 방식)을 채택하였으며, 여러개의 읽기전용 레이어로 구성되어 파일이 추가, 수정되면 새로운 레이어가 생성된다.
  * ex) ubuntu 이미지 = `A + B + C` , ubuntu 이미지로 만든 nginx 이미지 = `A + B + C + nginx` , nginx 이미지에서 개발한 app 이미지 = `A + B + C + nginx + source` , 수정한 app 이미지 = `A + B + C + nginx + source2`
![docker image layer](fig 10/fig10-4.png)
* 컨테이너 또한 레이어 방식인데, 기존의 이미지 레이어 위에 읽기전용이 아닌 읽기/쓰기 레이어를 추가한다.


## 3. 컨테이너 실행을 위한 코드

* 명령어의 경우 대문자와 소문자에 따라 다른 명령을 수행하므로 주의하도록 한다.
* `--detach , -d` : 컨테이너를 구동시키고, 컨테이너 ID를 보여준다.
* `--publish , -p` : 호스트와 컨테이너의 포트를 연결한다.
  * 컨테이너를 생성할 때 기본적으로 어떤 포트도 연결하지 않도록 되어있다.
  * 도커 바깥의 환경 또는 도커 내 이지만 연결되어 있지 않은 다른 컨테이너에서 포트를 이용가능하도록 만들 때 사용한다.
  * 대문자로 사용하는 경우 `--publish-all , -P` 모든 포트를 다룬다.
  * `-p 8080:80` : 컨테이너의 TCP 포트 80을 Docker 호스트의 포트 8080에 짝지어준다.
  * `-p 192.168.1.100:8080:80` : 컨테이너의 TCP 포트 80을 Docker 호스트의 포트 8080에 짝지어 호스트 IP 192.168.1.100에 연결한다.
  * `-p 8080:80/udp` : 컨테이너의 UDP 포트 80을 Docker 호스트의 포트 8080에 짝지어준다.
  * `-p 8080:80/tcp -p 8080:80/udp` : 컨테이너의 TCP, UDP 포트 80을 Docker 호스트의 TCP, UDP 포트 8080에 짝지어준다.
  * Git 에서 `publish`는 로컬 환경에 만든 저장소를 GitHub 서버에 게시하는 것을 말한다.
* `--volume , -v` : 호스트와 컨테이너의 디렉토리를 연결한다.
* `--env , -e` : 컨테이너 내에서 사용할 환경에 관련된 설정을 한다.
* `--name` : 컨테이너의 이름을 설정한다. 이름을 설정하지 않을 경우에는 유명인의 이름과 그를 꾸며주는 수식어를 조합하여 만든다.
![container name](fig 10/fig10-5.png)
* `--rm` : 프로세스를 종료할 때 작동 중이던 컨테이너가 자동으로 삭제된다.
* `--interactive , -i` : STDIN(Standard Input), 표준입력에 대한 포인터를 유지한다.
* `--tty , -t` : TTY(Teletypewriter). 컴퓨터에 연결되어 입출력에 사용되기도 했던 기계의 이름으로, CLI의 원조라고 할 수 있으며, `-t`는 가상의 TTY를 할당해준다.
* `-it` : `-i`, `-t`를 동시에 사용한 것으로 터미널을 통한 입력을 위한 옵션이다.
* `--link` : 다른 컨테이너와 연결시켜준다.


## 4. ubuntu

* 컨테이너 구동하기 `docker run ubuntu:latest`
![container ubuntu](fig 10/fig10-6.png)
* `run`을 명령하면 이미지가 있는지 확인한 후에 없을 경우 허브에서 pull 한 뒤 컨테이너를 생성하고 시작한다. (`pull+create+start`)
* 별다른 명령어를 입력하지 않았으므로 컨테이너는 바로 종료된다.
* `/bin/bash` 로 ubuntu bash 를 실행시킨다.
![container ubuntu](fig 10/fig10-7.png)
* 컨테이너의 내부에 들어가기 위해서 bash shell 을 실행하였다.
* shell 에서 키보드의 입력을 위해 `-it`를 명령한다.
* 프로세스가 종료되면 자동으로 컨테이너가 삭제되도록 `--rm`을 활성화한다.
![container ubuntu](fig 10/fig10-8.png)
* `exit`을 통해 shell을 종료한다. 컨테이너도 자동으로 삭제된다.
![container ubuntu](fig 10/fig10-9.png)


## 5. Tensorflow 컨테이너

* Tensorflow는 Python으로 만들어져 있으므로 파이썬 및 관련 패키지를 모두 설치하여야 한다.
* Tensorflow 공식 홈페이지에서는 파이썬, 도커, 코랩 등을 이용하여 Tensorflow를 실행시킬 수 있도록 매뉴얼을 제공한다.https://www.tensorflow.org/install
* 도커는 Tensorflow를 가상 환경에 독립적으로 설치 및 실행할 수 있도록 해준다.
* Tensorflow 프로그램은 자원과 호스트 머신(디렉토리 접근, GPU 사용, 인터넷 연결 등)을 공유할 수 있는 가상 환경에서 구동한다.
* 도커 이미지는 https://hub.docker.com/r/tensorflow/tensorflow/ 에서 확인할 수 있다.
* 도커를 통해 Linux 에서 Tensorflow의 GPU 지원을 받기 쉽다.
* Tensorflow는 다양한 tag를 지원하고 있다. GPU 지원, PYTHON3 지원 등 사용자가 원하는 환경에 맞춰 적당한 tag를 선택하여 다운받으면 된다. (https://hub.docker.com/r/tensorflow/tensorflow/tags/)
  * `latest`: The latest release of TensorFlow CPU binary image. Default.
  * `nightly`: Nightly builds of the TensorFlow image. (unstable)
  * `version`: Specify the version of the TensorFlow binary image, for example: 1.13.1
  * `devel`: Nightly builds of a TensorFlow master development environment. Includes TensorFlow source code.
  * `<tag>-gpu`: The specified tag release with GPU support.
  * `<tag>-py3`: The specified tag release with Python 3 support.
  * `<tag>-jupyter`: The specified tag release with Jupyter (includes TensorFlow tutorial notebooks)
  * 여러개의 tag를 한번에 사용할 수 있다. `docker pull tensorflow/tensorflow:latest-gpu-jupyter` `docker run -it --rm --runtime=nvidia tensorflow/tensorflow:latest-gpu-py3 python`

* Tensorflow 컨테이너 시작하기 `docker run -it --rm -p 8888:8888 tensorflow/tensorflow:latest`
![tensorflow](fig 10/fig10-10.png)

* Jupyter 에서 Tensorflow를 사용하기 위해 tag에 jupyter를 붙인 경우 `docker run -it --rm -p 8888:8888 tensorflow/tensorflow:latest-jupyter`
![tensorflow](fig 10/fig10-11.png)
* 아나콘다 프롬프트 실행 - `jupyter notebook list` 에서 해당하는 포트의 토큰을 입력한다.
![tensorflow](fig 10/fig10-12.png)
![tensorflow](fig 10/fig10-13.png)


다음에 할 것

* Tensorflow 실행을 로컬 디스크 내에서 할 것
* 랩실 도커 서버 만들기 https://yongho1037.tistory.com/693
* MySQL 컨테이너와 WordPress 컨테이너 연결하기


참조

http://pyrasis.com/book/DockerForTheReallyImpatient/Chapter07/07

https://stackoverflow.com/questions/22111060/what-is-the-difference-between-expose-and-publish-in-docker

https://subicura.com/2017/01/19/docker-guide-for-beginners-1.html

https://subicura.com/2017/01/19/docker-guide-for-beginners-2.html

https://docs.docker.com/engine/reference/commandline/run/

https://docs.docker.com/config/containers/container-networking/

https://ko.wikipedia.org/wiki/사용자_데이터그램_프로토콜

https://ko.wikipedia.org/wiki/클라이언트_서버_모델

https://yongho1037.tistory.com/693

https://mug896.github.io/bash-shell/tty.html

https://www.tensorflow.org/install/docker

https://www.joinc.co.kr/w/man/12/docker/Guide/DataWithContainer
